In [1]:
import pandas as pd
import numpy as np
import os

import re
import emoji
from konlpy.tag import Okt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader -> datasets 라이브러리랑 충돌

c:\Users\ehddl\AppData\Local\pypoetry\Cache\virtualenvs\sns-categorizer-wO1G7-CE-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/")

In [3]:
new_data = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data.parquet")
new_data = new_data[['media_cn']]

In [4]:
new_data.dropna(inplace=True)

In [5]:
def clean_text(text):
    if not isinstance(text, str):
        return ''
    
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [6]:
new_data['media_cn_cleaned'] = new_data['media_cn'].apply(clean_text)

In [7]:
new_data = new_data[~new_data.apply(lambda row: row.astype(str).str.strip().eq('').any(), axis=1)]

In [8]:
new_data

,media_cn,media_cn_cleaned
0,YJM* will Drop a new album in the hottest summ...,yjm will drop a new album in the hottest summer
1,마지막에 무서운 사진이 나옵니다….😱\n이태리 공항에서 마무리 도너츠 까지\n맛있으...,마지막에 무서운 사진이 나옵니다 이태리 공항에서 마무리 도너츠 까지 맛있으면 0칼로...
2,…\n안녕하세요.\n감 미사 입니다.\n가을이네요……. \n저 잘 익었습니다.\n빨...,안녕하세요 감 미사 입니다 가을이네요 저 잘 익었습니다 빨리 수확해 주지 않으면 썩...
3,푸드트럭으로 오는데 여기 순대 너무 맛있어!!!\n야채가득!!! 여기 순대가 은근 ...,푸드트럭으로 오는데 여기 순대 너무 맛있어 야채가득 여기 순대가 은근 맛있어서 중독...
4,#제품협찬\n\n매일매일 정성으로 직접 만드는 #흑임자인절미\n청년떡집 명례헌 시그...,제품협찬 매일매일 정성으로 직접 만드는 흑임자인절미 청년떡집 명례헌 시그니처 제품 ...
...,...,...
19615,KiiiKiii with #SSG랜더스❤️\n\n#KiiiKiii #키키 \n#SU...,kiiikiii with ssg랜더스 kiiikiii 키키 sui 수이
19616,#협찬 침구 바꿨더니 호텔같대요🩵\n\n잠깐 사는 군인관사이지만 깔꿈하게 살구싶어....,협찬 침구 바꿨더니 호텔같대요 잠깐 사는 군인관사이지만 깔꿈하게 살구싶어 나중에 이...
19617,#공구예고 duri.official \n\n📢공구 안하는 브랜드 두리\n켈리맘이 최...,공구예고 duriofficial 공구 안하는 브랜드 두리 켈리맘이 최초로 공구 최저...
19618,• clearance sale last day ! • \n\n금일 밤 11시 55분...,clearance sale last day 금일 밤 11시 55분 클리어런스 세일이...


In [11]:
num_labels = 26
new = new_data.drop(['media_cn'], axis=1)
predict_dataset = Dataset.from_pandas(new)

single model prediction

In [51]:
model_name_1 = "kykim/bert-kor-base"
model_path_1 = "finetune-bert-kor"

model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
model1_tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
model1.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [36]:
model_name_2 = "BM-K/KoSimCSE-roberta"
model_path_2 = "finetune-BM-K"

model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
model2_tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
model2.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [37]:
new = new_data.drop(['media_cn'], axis=1)

In [38]:
predict_dataset = Dataset.from_pandas(new)

In [52]:
def tokenize_fn_for_prediction(ex):
    # label 칼럼이 없으므로 text 컬럼만 토큰화 진행
    return model1_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128) # max_length 조정 (128->512)

predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"]) # 라벨은 없으므로 제외

Map: 100%|██████████| 18981/18981 [00:01<00:00, 10980.10 examples/s]


In [39]:
# def tokenize_fn_for_prediction(ex):
#     tokenized_output = model2_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)
#     if "token_type_ids" not in tokenized_output:
#         tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
#     return tokenized_output

# predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
# predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids"]) # 라벨은 없으므로 제외

Map: 100%|██████████| 18981/18981 [00:01<00:00, 12261.69 examples/s]


In [ ]:
prediction_args = TrainingArguments(
    output_dir="./prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False, # 학습은 안 함
    do_predict=True, # 예측만 수행
    report_to="none", # W&B 등 로깅 비활성화
    disable_tqdm=False, # 진행률 바 표시 (기본값 True)
)

# Trainer 인스턴스 생성
trainer = Trainer(model=model1, args=prediction_args) # 모델 변경

# 예측 수행
predictions_output = trainer.predict(predict_dataset)

# predictions_output 객체에서 로짓 추출
logits = predictions_output.predictions
print(f"\n예측된 로짓 형태: {logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()

# 가장 높은 확률을 가진 라벨의 인덱스 추출
predicted_class_indices = np.argmax(logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in predicted_class_indices]


예측된 로짓 형태: (18981, 26)


ensemble predict

In [17]:
# --- 0.4. 각 모델에 맞는 토크나이저 로드 및 텍스트 토큰화 ---

# kykim 모델용 토크나이저 및 데이터셋
model_name_1 = "kykim/bert-kor-base"
model_path_1 = "finetune-bert-kor"
model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
model1.eval()

tokenizer_kykim = AutoTokenizer.from_pretrained(model_name_1)
def tokenize_fn_kykim_for_predict(examples):
    return tokenizer_kykim(examples["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)

predict_dataset_kykim_tokenized = predict_dataset.map(tokenize_fn_kykim_for_predict, batched=True)
predict_dataset_kykim_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])

# KoSimCSE 모델용 토크나이저 및 데이터셋
model_name_2 = "BM-K/KoSimCSE-roberta"
model_path_2 = "finetune-BM-K"

model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
model2.eval()

tokenizer_kosimcse = AutoTokenizer.from_pretrained(model_name_2)
def tokenize_fn_kosimcse_for_predict(examples):
    tokenized_output = tokenizer_kosimcse(examples["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)
    if "token_type_ids" not in tokenized_output:
        tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
    return tokenized_output

predict_dataset_kosimcse_tokenized = predict_dataset.map(tokenize_fn_kosimcse_for_predict, batched=True)
predict_dataset_kosimcse_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids"])

# --- 2. Trainer를 사용하여 각 모델의 예측 로짓(Logits) 얻기 ---
# 더미 TrainingArguments (예측용이므로 대부분의 인자는 중요하지 않음)
prediction_args = TrainingArguments(
    output_dir="./voting_prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False,
    do_predict=True,
    report_to="none", # W&B 등 로깅 비활성화
    remove_unused_columns=False
)

# Model 1 Prediction
trainer1 = Trainer(model=model1, args=prediction_args)
predictions1 = trainer1.predict(predict_dataset_kykim_tokenized)
logits1 = predictions1.predictions # numpy array 형태의 로짓

# Model 2 Prediction
trainer2 = Trainer(model=model2, args=prediction_args)
predictions2 = trainer2.predict(predict_dataset_kosimcse_tokenized)
logits2 = predictions2.predictions


Map: 100%|██████████| 18282/18282 [00:01<00:00, 12330.75 examples/s]


In [18]:
# --- 3. 로짓(Logits) 또는 확률(Probabilities) 평균내기 ---
# 소프트 보팅은 로짓 또는 softmax 확률을 평균낼 수 있음 (softmax는 비선형 변환이므로 평균 내기 전에 로짓 상태에서 평균 내는 것이 좋음)
# 일반적으로 로짓을 평균내는 것이 더 안정적

# 로짓 평균
ensemble_logits = (logits1 + logits2) / 2 
print(f"\n예측된 로짓 형태: {ensemble_logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(ensemble_logits), dim=-1).numpy()

# 평균 로짓에서 최종 예측 클래스 선택
ensemble_predictions = np.argmax(ensemble_logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in ensemble_predictions]


예측된 로짓 형태: (18282, 26)


In [19]:
new['predicted_label'] = predicted_label_names
new['predicted_label'] = new['predicted_label'].astype(int)

In [20]:
new

,media_cn_cleaned,predicted_label
0,yjm will drop a new album in the hottest summer,19
1,마지막에 무서운 사진이 나옵니다 이태리 공항에서 마무리 도너츠 까지 맛있으면 0칼로...,17
2,안녕하세요 감 미사 입니다 가을이네요 저 잘 익었습니다 빨리 수확해 주지 않으면 썩...,19
3,푸드트럭으로 오는데 여기 순대 너무 맛있어 야채가득 여기 순대가 은근 맛있어서 중독...,18
4,제품협찬 매일매일 정성으로 직접 만드는 흑임자인절미 청년떡집 명례헌 시그니처 제품 ...,24
...,...,...
19615,kiiikiii with ssg랜더스 kiiikiii 키키 sui 수이,14
19616,협찬 침구 바꿨더니 호텔같대요 잠깐 사는 군인관사이지만 깔꿈하게 살구싶어 나중에 이...,25
19617,공구예고 duriofficial 공구 안하는 브랜드 두리 켈리맘이 최초로 공구 최저...,9
19618,clearance sale last day 금일 밤 11시 55분 클리어런스 세일이...,23


In [21]:
label_map = {
 'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '다이어트/건강보조식품': 4,
 '만화/애니/툰': 5,
 '문구/완구': 6,
 '미술/디자인': 7,
 '반려동물': 8,
 '베이비/키즈': 9,
 '뷰티': 10,
 '브랜드공식계정': 11,
 '사진/영상': 12,
 '셀럽': 13,
 '스포츠': 14,
 '시사': 15,
 '엔터테인먼트': 16,
 '여행/관광': 17,
 '유명장소/핫플': 18,
 '일상': 19,
 '자동차/모빌리티': 20,
 '짤/밈': 21,
 '취미': 22,
 '패션': 23,
 '푸드': 24,
 '홈/리빙': 25
}

inv_label_map = {v: k for k, v in label_map.items()}
new['single_label'] = new['predicted_label'].map(inv_label_map)

In [22]:
new.to_excel("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/new_3.xlsx")